# Vortex precession in a 2D parabolic trap
### Ashton Bradley

# Introduction
In this example we simulate the precession of a single quantum vortex in a harmonic trap, and compare with the analytical result found by Fetter.

As simple model of this process, we start with the damped GPE
$$
i\hbar\partial_t\psi = (1-i\gamma)(L-\mu)\psi
$$
where the GP operator is defined as
$$
L\psi\equiv \left(-\frac{\hbar^2\nabla^2}{2m}+V(\mathbf{r})+g|\psi|^2\right)\psi
$$
and as usual the S-wave interaction parameter is $g=4\pi\hbar^2 a/m$ for S-wave scattering
lenght $a$.

The trap is chosen to be cylindrically symmetric
$$
V(\mathbf{r})=\frac{m\omega_\perp^2}{2}(x^2 + y^2)
$$
The number $\gamma\ll 1$ describes irreversable interactons between condensate
and noncondensate atoms, inducing condensate growth.

# Loading the package
First, we load some useful packages, setting defaults for `Plots`.

In [ ]:
using Plots, LaTeXStrings
gr(titlefontsize=12,size=(500,300),transpose=true,colorbar=false)

Now load `FourierGPE`

In [ ]:
using FourierGPE

In this example, we work in oscillator units. The units of length and time are
$a_\perp=\sqrt{\hbar/m\omega_\perp}$ and $1/\omega_\perp$ respectively.

# Initialize simulation
Initialize default sim with domain and grid parameters

In [ ]:
L = (20.0,20.0)
N = (128,128)
sim = Sim(L,N)
@unpack_Sim sim;

# Declare the potential

In [ ]:
import FourierGPE.V
V(x,y,t)::Float64 = 0.5*(x^2 + y^2)

# Thomas-Fermi initial state

In [ ]:
ψ0(x,y,μ,g) = sqrt(μ/g)*sqrt(max(1.0-V(x,y,0.0)/μ,0.0)+im*0.0)
x,y = X

μ = 25.0
ψi = ψ0.(x,y',μ,g)
ϕi = kspace(ψi,sim)
@pack_Sim! sim;

# Imaginary-time evolution in k space

In [ ]:
@time sol = runsim(sim);

Let's pull out the final state and verify that it is indeed the ground state of the harmonic trap:

In [ ]:
ϕg = sol[end]
ψg = xspace(ϕg,sim)
showpsi(x,y,ψg)

# Time dynamics: precession of an off-axis vortex
We can use `VortexDistributions` to imprint a vortex off axis and test vortex precession rate according to the GPE.

In [ ]:
using VortexDistributions

# Initial state
We imprint a vortex inside the Thomas-Fermi radius

In [ ]:
healing(x,y,μ,g) = 1/sqrt(g*abs2(ψ0(x,y,μ,g)))
Rtf = sqrt(2*μ)
rv = 0.5*Rtf
xv,yv,cv = rv, 0.0, 1
pv1 = PointVortex(xv,yv,cv) # coordinates and charge for vortex
ξv = healing(xv,yv,μ,g) # local healing length at the vortex
v1 = ScalarVortex(ξv,pv1) # define scalar GPE vortex with local healing length

ψ1 = Torus(copy(ψg),x,y) # methods in VortexDistributions require type conversion
vortex!(ψ1,v1) # phase/density imprint vortex
ψv = ψ1.ψ #pull out the new wavefunction
showpsi(x,y,ψv)

# Precession frequency
In the Thomas-Fermi regime, precession frequency is given analytically in terms of the
healing length $\xi$ by (see e.g. [Fetter JLTP 2010](https://dx.doi.org/10.1007/s10909-010-0202-7)):
$$
\begin{align}
\Omega_m &= \frac{3}{2R_{\rm TF}^2}\log{\left(\frac{R_{\rm TF}}{\xi\sqrt{2}}\right)}\\
\Omega_v &= \frac{\Omega_m}{1-r_v^2/R_{\rm TF}^2}
\end{align}
$$
For our parameters this is

In [ ]:
ξ = 1/sqrt(μ)
Ωm = 3*log(Rtf/ξ/sqrt(2))/2/Rtf^2
Ωv = Ωm/(1-rv^2/Rtf^2)

or a vortex precession period of

In [ ]:
Tv = 2*π/Ωv

# Set simulation parameters
Let's evolve for one period of Hamiltonian dynamics, as predicted by the Thomas-Fermi analysis:

In [ ]:
γ = 0.0
tf = Tv
t = LinRange(ti,tf,Nt)
ϕi = kspace(ψv,sim)
@pack_Sim! sim; # write over previous sim  and reuse

# Evolve in k space

In [ ]:
solv = runsim(sim);

Pull out a state after some evolution

In [ ]:
ϕf = solv[100]
ψf = xspace(ϕf,sim)
showpsi(x,y,ψf)

We can trim the last few frames to show one orbit

In [ ]:
anim = @animate for i=1:Nt-6
    ψ = xspace(solv[i],sim)
    showpsi(x,y,ψ)
end;

and save the animation to the [media folder](../../media/vortex.gif)

In [ ]:
gif(anim,"./media/vortex.gif",fps=30)

The simulation gives a precession frequency within about $\sim 10\%$ of the analytical result.